In [7]:
import pickle

with open("/Users/christoffer/Downloads/spots.pkl", "rb") as f:
    obj = pickle.load(f)

In [8]:
# Unpack and inspect
spots = obj["spots"]
polygons = obj["polygons"]

spots.keys(), polygons.keys()


(dict_keys(['x', 'y', 'gene', 'section']),
 dict_keys(['02A', '06A', '10A', '14A', '18A', '02B', '06B', '10B', '14B', '18B', '02C', '06C', '10C', '14C', '18C', '02D', '06D', '10D', '14D', '18D', '02E', '06E', '10E', '14E', '18E']))

In [9]:
# Convert spots to a DataFrame for easier wrangling
import pandas as pd

spots_df = pd.DataFrame(spots)
spots_df.head()


,x,y,gene,section
0,16861,19460,Fezf1,02A
1,16892,19535,Lmo2,02A
2,16901,19544,Fezf2,02A
3,16915,19474,Efna2,02A
4,16922,18476,Fezf2,02A


In [13]:
len(spots_df.gene.unique())

119

In [11]:
# Export per-section CSVs for Baysor
from pathlib import Path

out_dir = Path("/Volumes/processing2/nature-dev-mouse-reanalysis/data")
out_dir.mkdir(parents=True, exist_ok=True)

export_cols = ["x", "y", "gene"]

for section, df_sec in spots_df.groupby("section", sort=True):
    section_dir = out_dir / f"section_{section}"
    section_dir.mkdir(parents=True, exist_ok=True)
    out_path = section_dir / "spots.csv"
    df_sec[export_cols].to_csv(out_path, index=False)

out_dir


PosixPath('/Volumes/processing2/nature-dev-mouse-reanalysis/data')

In [15]:
# Export polygons per section (as JSON)
import json
import numpy as np
from pathlib import Path

poly_out_dir = Path("/Volumes/processing2/nature-dev-mouse-reanalysis/data")
poly_out_dir.mkdir(parents=True, exist_ok=True)

def to_jsonable(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    if isinstance(obj, np.generic):
        return obj.item()
    raise TypeError(f"Object of type {type(obj).__name__} is not JSON serializable")

# Expect polygons as a dict keyed by section name/id
for section, poly in polygons.items():
    section_dir = poly_out_dir / f"section_{section}"
    section_dir.mkdir(parents=True, exist_ok=True)
    out_path = section_dir / "polygons.json"
    with open(out_path, "w") as f:
        json.dump(poly, f, default=to_jsonable)

poly_out_dir


TypeError: Object of type ndarray is not JSON serializable